In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marathi-summarization-dataset/marathi-marathi_val.jsonl
/kaggle/input/marathi-summarization-dataset/marathi-marathi_train.jsonl
/kaggle/input/marathi-summarization-dataset/marathi-marathi_test.jsonl
/kaggle/input/summarisaton-mar-emg/mar_eng_sum.csv
/kaggle/input/english-summarization-dataset/test_final_eng.csv
/kaggle/input/english-summarization-dataset/val_final_eng.csv
/kaggle/input/english-summarization-dataset/train_final_eng.csv


In [4]:
import pandas as pd
import json

# Path to the JSON Lines file
file_path = ["/kaggle/input/marathi-summarization-dataset/marathi-marathi_test.jsonl", "/kaggle/input/marathi-summarization-dataset/marathi-marathi_train.jsonl","/kaggle/input/marathi-summarization-dataset/marathi-marathi_val.jsonl"]


# Initialize an empty list to hold the JSON objects
data = []

# Open the file and read line by line
for i in range(3):
    data.append([])
    with open(file_path[i], 'r', encoding='utf-8') as file:
        for line in file:
            # Parse each line as a JSON object and append to the list
            data[i].append(json.loads(line))
    

# Convert the list of JSON objects into a DataFrame
df_mr = pd.DataFrame(data[1])
df_test_cs_mar = pd.DataFrame(data[0])
df_val_cs_mar = pd.DataFrame(data[2])

# Display the first few rows of the DataFrame to check if it's loaded correctly
print(len(df_mr))
print(len(df_test_cs_mar))
print(len(df_val_cs_mar))

10558
1188
1254


In [5]:
df_val_cs_mar.columns

Index(['source_url', 'target_url', 'text', 'summary'], dtype='object')

In [6]:
import pandas as pd
df_en=pd.read_csv("/kaggle/input/english-summarization-dataset/train_final_eng.csv")
df_en.columns

Index(['source_url', 'target_url', 'text', 'summary'], dtype='object')

In [7]:
df_en_test=pd.read_csv("/kaggle/input/english-summarization-dataset/test_final_eng.csv")
df_en_test.columns
df_en_val=pd.read_csv("/kaggle/input/english-summarization-dataset/val_final_eng.csv")
df_en_val.columns

Index(['source_url', 'target_url', 'text', 'summary'], dtype='object')

In [8]:
import pandas as pd
df=pd.read_csv("/kaggle/input/summarisaton-mar-emg/mar_eng_sum.csv")
df.columns

Index(['url', 'text_mr', 'summary_en', 'summary_mr', 'text_en'], dtype='object')

In [9]:
print("NaNs in df_en 'text':", df_en['text'].isnull().sum())
print("NaNs in df_mr 'text':", df_mr['text'].isnull().sum())

NaNs in df_en 'text': 0
NaNs in df_mr 'text': 0


In [10]:
df_en = df_en.sample(n=20000, random_state=42)

In [11]:
df_en.columns

Index(['source_url', 'target_url', 'text', 'summary'], dtype='object')

In [12]:
print(df_en.isna().sum())

source_url    0
target_url    0
text          0
summary       0
dtype: int64


In [13]:
# Label relevant pairs with '1'
df_en['relevance'] = 1
df_mr['relevance'] = 1
# Shuffle texts within each DataFrame to create irrelevant pairs
# Reset index before sampling to ensure alignment
df_en_reset = df_en.reset_index(drop=True)
df_mr_reset = df_mr.reset_index(drop=True)

df_en_irrelevant = df_en_reset.copy().assign(text=df_en_reset['text'].sample(frac=1).reset_index(drop=True), relevance=0)
df_mr_irrelevant = df_mr_reset.copy().assign(text=df_mr_reset['text'].sample(frac=1).reset_index(drop=True), relevance=0)

In [14]:
# Combine relevant and irrelevant pairs for each language
df_en_combined = pd.concat([df_en, df_en_irrelevant])
df_mr_combined = pd.concat([df_mr, df_mr_irrelevant])

# Combine English and Marathi data
df_combined = pd.concat([df_en_combined, df_mr_combined], ignore_index=True)

In [15]:
# Check for NaNs after combining
print("NaNs in combined 'text':", df_combined['text'].isnull().sum())

NaNs in combined 'text': 0


In [16]:
df_combined.columns

Index(['source_url', 'target_url', 'text', 'summary', 'relevance'], dtype='object')

In [17]:
# Shuffle the combined DataFrame to ensure a random distribution of data
df_combined = df_combined.sample(frac=1).reset_index(drop=True)

In [18]:
print(len(df_combined))

61116


In [19]:
print(df_combined.isna().sum())

source_url    0
target_url    0
text          0
summary       0
relevance     0
dtype: int64


In [20]:
import pandas as pd

# Assuming df_en and df_mr are your English and Marathi DataFrames respectively
# Sampling 10,000 rows from each DataFrame
df_en_sample = df_en.sample(n=10000, random_state=42)  # English DataFrame
df_mr_sample = df_mr.sample(n=10000, random_state=42)  # Marathi DataFrame

In [21]:
# Reset index to align the samples
df_en_sample.reset_index(drop=True, inplace=True)
df_mr_sample.reset_index(drop=True, inplace=True)

# Create cross-lingual irrelevant pairs
# English text with Marathi summary
df_en_mr_irrelevant = pd.DataFrame({
    'text': df_en_sample['text'],
    'summary': df_mr_sample['summary'],
    'relevance': 0  # Mark as irrelevant
})

# Marathi text with English summary
df_mr_en_irrelevant = pd.DataFrame({
    'text': df_mr_sample['text'],
    'summary': df_en_sample['summary'],
    'relevance': 0  # Mark as irrelevant
})

In [22]:
def create_relevance_dataset(df):
    # Correct Pairings with relevance = 1
    correct_pairs_en = df[['summary_en', 'text_en']].rename(columns={'summary_en': 'summary', 'text_en': 'text'})
    correct_pairs_en['relevance'] = 1
    
    correct_pairs_mr = df[['summary_mr', 'text_mr']].rename(columns={'summary_mr': 'summary', 'text_mr': 'text'})
    correct_pairs_mr['relevance'] = 1

    # Incorrect Pairings with relevance = 0
    # Shuffle the texts and pair with the original summaries
    wrong_texts_en = df['text_en'].sample(frac=1).reset_index(drop=True)
    incorrect_pairs_en = pd.DataFrame({'summary': df['summary_en'], 'text': wrong_texts_en})
    incorrect_pairs_en['relevance'] = 0

    wrong_texts_mr = df['text_mr'].sample(frac=1).reset_index(drop=True)
    incorrect_pairs_mr = pd.DataFrame({'summary': df['summary_mr'], 'text': wrong_texts_mr})
    incorrect_pairs_mr['relevance'] = 0

    # Combine all pairs
    combined_dataset = pd.concat([correct_pairs_en, correct_pairs_mr, incorrect_pairs_en, incorrect_pairs_mr])
    
    # Shuffle the combined dataset
    combined_dataset = combined_dataset.sample(frac=1).reset_index(drop=True)

    return combined_dataset

# Create the relevance dataset
relevance_dataset = create_relevance_dataset(df)

In [23]:
def create_relevance_dataset_2(df):
    # Correct Pairings with relevance = 1
    correct_pairs_en = df[['summary_en', 'text_mr']].rename(columns={'summary_en': 'summary', 'text_mr': 'text'})
    correct_pairs_en['relevance'] = 1
    
    correct_pairs_mr = df[['summary_mr', 'text_en']].rename(columns={'summary_mr': 'summary', 'text_en': 'text'})
    correct_pairs_mr['relevance'] = 1

    # Incorrect Pairings with relevance = 0
    # Shuffle the texts and pair with the original summaries
    wrong_texts_en = df['text_en'].sample(frac=1).reset_index(drop=True)
    incorrect_pairs_en = pd.DataFrame({'summary': df['summary_mr'], 'text': wrong_texts_en})
    incorrect_pairs_en['relevance'] = 0

    wrong_texts_mr = df['text_mr'].sample(frac=1).reset_index(drop=True)
    incorrect_pairs_mr = pd.DataFrame({'summary': df['summary_en'], 'text': wrong_texts_mr})
    incorrect_pairs_mr['relevance'] = 0

    # Combine all pairs
    combined_dataset = pd.concat([correct_pairs_en, correct_pairs_mr, incorrect_pairs_en, incorrect_pairs_mr])
    
    # Shuffle the combined dataset
    combined_dataset = combined_dataset.sample(frac=1).reset_index(drop=True)

    return combined_dataset

# Create the relevance dataset
relevance_dataset_2 = create_relevance_dataset_2(df)

In [24]:
print(relevance_dataset['relevance'][1],relevance_dataset['summary'][1])

0 सौदी अरेबिया आणि इराणमधला संघर्ष फार जुना आहे. वर्षानुवर्षं हे देश एकमेकांचे विरोधक आहेत. पण गेल्या काही दिवसांमध्ये या दोन देशांतला तणाव वाढला आहे.


In [25]:
print(len(relevance_dataset))

4684


In [26]:
print(len(relevance_dataset_2)), relevance_dataset_2['summary']

4684


(None,
 0       US President Donald Trump's Twitter account is...
 1       Nearly three decades, 850 witnesses, more than...
 2       Indian Prime Minister Narendra Modi has always...
 3       फ्रान्समधले शेतकरी सध्या एका भलत्याच प्रश्नाने...
 4       Unknown to most Iranians until Friday, when he...
                               ...                        
 4679    अमेरिकेसाठी अफगाणिस्तानातलं युद्ध हे सगळ्यांत ...
 4680    India's government has decided to sell a contr...
 4681    European Council President Donald Tusk has sai...
 4682    A coronavirus vaccine developed by the Univers...
 4683    If you are reading the headlines in the Indian...
 Name: summary, Length: 4684, dtype: object)

In [27]:
print(df_combined['summary'])

0        A feminist video-games critic has cancelled a ...
1        Appropriate Adult, ITV1's dramatisation of the...
2        A new report says that oil and gas producing c...
3        Assembly member Nick Ramsay has said he has pu...
4        Here is the full list of winners for the 2014 ...
                               ...                        
61111    इंग्लंडच्या सैन्यात आता गुरखा स्त्रिया असतील. ...
61112    A teenager who killed a schoolboy after paying...
61113    The police watchdog should be given greater po...
61114    राष्ट्रीय किसान महासंघाने देशव्यापी शेतकरी संप...
61115    चीनच्या सर्वकालीन महान देशप्रमुखांमध्ये दिग्गज...
Name: summary, Length: 61116, dtype: object


In [28]:
df_final = pd.concat([relevance_dataset, relevance_dataset_2, df_en_mr_irrelevant, df_mr_en_irrelevant, df_combined])

In [29]:
df_final.columns, len(df_final)

(Index(['summary', 'text', 'relevance', 'source_url', 'target_url'], dtype='object'),
 90484)

In [30]:
from huggingface_hub import notebook_login
notebook_login()

In [31]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
print(df_final.isna().sum())

summary           0
text              0
relevance         0
source_url    29368
target_url    29368
dtype: int64


In [31]:
df_f = df_final[['summary','text','relevance']]

In [32]:
print(df_f.isna().sum())

summary      0
text         0
relevance    0
dtype: int64


In [33]:
relevant_count = df_f[df_f['relevance'] == 1].shape[0]
irrelevant_count = df_f[df_f['relevance'] == 0].shape[0]

# Print the results
print(f"Number of relevant summaries: {relevant_count}")
print(f"Number of irrelevant summaries: {irrelevant_count}")

Number of relevant summaries: 35242
Number of irrelevant summaries: 55242


In [34]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(df_f, test_size=0.1, random_state=42)  # Adjust test_size as needed

In [36]:
train_data.columns

Index(['summary', 'text', 'relevance'], dtype='object')

In [37]:
!pip install datasets==2.15

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.0
    Uninstalling fsspec-2024.3.0:
      Successfully uninstalled fsspec-2024.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Foun

In [ ]:
def tokenize_function(row):
    return tokenizer(row['text'], row['summary'], padding='max_length', truncation=True, max_length=512)

# Apply tokenization to each row of the DataFrame
tokenized_data_train = train_data.apply(tokenize_function, axis=1)
tokenized_data_val = val_data.apply(tokenize_function, axis=1)

In [19]:
from transformers import AutoTokenizer
import pandas as pd
import torch

# Assuming tokenizer is defined, for example:
# tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

def batch_tokenize_dataframe(df, tokenizer, batch_size=16):
    # Initialize lists to hold tokenized data
    tokenized_batches = {
        "input_ids": [],
        "attention_mask": [],
        # Include "token_type_ids" if your model needs them
    }

    # Process dataframe in batches
    for start_idx in range(0, len(df), batch_size):
        end_idx = start_idx + batch_size
        batch = df.iloc[start_idx:end_idx]
        
        # Tokenize batch
        tokenized_output = tokenizer(
            batch["text"].tolist(), 
            batch["summary"].tolist(), 
            padding="max_length", 
            truncation=True, 
            max_length=512, 
            return_tensors="pt"
        )
        
        # Extend the lists
        for key in tokenized_batches.keys():
            tokenized_batches[key].append(tokenized_output[key])

    # Concatenate all batches
    for key in tokenized_batches.keys():
        tokenized_batches[key] = torch.cat(tokenized_batches[key], dim=0)

    # Convert to DataFrame (optional, depending on how you plan to use the tokenized data)
    tokenized_data_df = pd.DataFrame({
        'input_ids': tokenized_batches['input_ids'].tolist(),
        'attention_mask': tokenized_batches['attention_mask'].tolist(),
        # Add other fields if necessary
    })

    return tokenized_data_df


In [ ]:
# Apply the batch tokenization function to your dataframes
tokenized_data_train_df = batch_tokenize_dataframe(train_data, tokenizer)
tokenized_data_val_df = batch_tokenize_dataframe(val_data, tokenizer)

In [39]:
from torch.utils.data import Dataset, DataLoader
import torch

class RelevanceDataset(Dataset):
    def __init__(self, tokenized_data_df, original_df):
        self.tokenized_data_df = tokenized_data_df
        self.labels = original_df['relevance'].values  # Assuming 'relevance' is the column with labels

    def __len__(self):
        return len(self.tokenized_data_df)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_data_df.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Initialize the dataset
train_dataset = RelevanceDataset(tokenized_data_train_df, train_data)

# Example usage with DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# To test if your DataLoader works and inspect a batch
for batch in train_loader:
    print(batch)
    break  # Just to check the first batch

{'input_ids': tensor([[   104,   1165, 122392,  ...,   1155,    121,    105],
        [   104,  43465,   5100,  ...,      0,      0,      0],
        [   104,   1129,   8056,  ...,      0,      0,      0],
        ...,
        [   104,  32794,  91093,  ...,  15293,    138,    105],
        [   104,  47028,   1227,  ...,      0,      0,      0],
        [   104,   1265,  11109,  ...,   1370,    121,    105]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0])}


In [40]:
val_dataset = RelevanceDataset(tokenized_data_val_df, val_data)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

# To test if your DataLoader works and inspect a batch
for batch in val_loader:
    print(batch)
    break  # Just to check the first batch

{'input_ids': tensor([[  104,  1129, 61625,  ...,     0,     0,     0],
        [  104, 15188,  1855,  ...,  1155,   121,   105],
        [  104,  7056,   165,  ..., 13282,   121,   105],
        ...,
        [  104, 40559,  1108,  ..., 15154,   121,   105],
        [  104,  1129,  8491,  ...,     0,     0,     0],
        [  104, 27036, 51679,  ...,  1155,   121,   105]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1])}


In [41]:
print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

Training dataset size: 81435
Validation dataset size: 9049


In [42]:
model.gradient_checkpointing_enable()

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [44]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=f'MuRIL_relevance',
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_total_limit=1,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    push_to_hub=True,
    gradient_accumulation_steps = 32,
)

2024-04-07 05:33:08.574882: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 05:33:08.574972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 05:33:08.703962: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [45]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [46]:
trainer.train(resume_from_checkpoint=True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
9,0.060700,0.026391


TrainOutput(global_step=390, training_loss=0.003917516500521929, metrics={'train_runtime': 5803.3633, 'train_samples_per_second': 140.324, 'train_steps_per_second': 0.067, 'total_flos': 2.1051462917142528e+17, 'train_loss': 0.003917516500521929, 'epoch': 9.98})

In [47]:
trainer.push_to_hub(commit_message="Training complete", tags="classification")

CommitInfo(commit_url='https://huggingface.co/october-sd/MuRIL_relevance/commit/db0528c7a6476e96839fb7095f7a8a6f3c1d06dc', commit_message='Training complete', commit_description='', oid='db0528c7a6476e96839fb7095f7a8a6f3c1d06dc', pr_url=None, pr_revision=None, pr_num=None)

In [48]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

def generate_relevance_score(model_path, text, summary):
    """
    Generate a relevance score for a given text and summary pair using a fine-tuned MuRIL model.

    Args:
        model_path (str): Path to the fine-tuned MuRIL model.
        text (str): The input text.
        summary (str): The corresponding summary.

    Returns:
        float: The relevance score.
    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
    model = AutoModelForSequenceClassification.from_pretrained(model_path)

    # Ensure the model is in evaluation mode
    model.eval()

    # Tokenize the text and summary
    inputs = tokenizer(text, summary, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)

    # Assuming the second column of the output represents the "relevance" class
    relevance_score = probabilities[:, 1].item()  # Convert to Python float

    return relevance_score

# Example usage
model_path = "october-sd/MuRIL_relevance"
text = relevance_dataset['text'][2]
summary = relevance_dataset['summary'][2]

relevance_score = generate_relevance_score(model_path, text, summary)
print(f"Relevance Score: {relevance_score:.2f}")

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

Relevance Score: 0.98


In [35]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Ensure a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model_name = "october-sd/MuRIL_relevance"
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
model.eval()  # Put the model in evaluation mode

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [41]:
tokenized_data_val_df = batch_tokenize_dataframe(df_f_v, tokenizer)

In [43]:
from torch.utils.data import Dataset, DataLoader
import torch

class RelevanceDataset(Dataset):
    def __init__(self, tokenized_data_df, original_df):
        self.tokenized_data_df = tokenized_data_df
        self.labels = original_df['relevance'].values  # Assuming 'relevance' is the column with labels

    def __len__(self):
        return len(self.tokenized_data_df)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_data_df.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [45]:
val_dataset = RelevanceDataset(tokenized_data_val_df, df_f_v)

In [46]:
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Prepare lists to store outputs
true_labels = []
predictions = []

# Inference
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for batch in val_loader:
        # Move batch to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass, calculate predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        pred_labels = torch.argmax(logits, dim=1)

        # Store predictions and true labels
        predictions.extend(pred_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9956
Precision: 0.9956
Recall: 0.9956
F1 Score: 0.9956


In [26]:
# Label relevant pairs with '1'
df_en_test['relevance'] = 1
df_val_cs_mar['relevance'] = 1
# Shuffle texts within each DataFrame to create irrelevant pairs
# Reset index before sampling to ensure alignment
df_en_reset = df_en_test.reset_index(drop=True)
df_mr_reset = df_val_cs_mar.reset_index(drop=True)

df_en_irrelevant = df_en_reset.copy().assign(text=df_en_reset['text'].sample(frac=1).reset_index(drop=True), relevance=0)
df_mr_irrelevant = df_mr_reset.copy().assign(text=df_mr_reset['text'].sample(frac=1).reset_index(drop=True), relevance=0)

In [27]:
# Combine relevant and irrelevant pairs for each language
df_en_combined = pd.concat([df_en_test, df_en_irrelevant])
df_mr_combined = pd.concat([df_val_cs_mar, df_mr_irrelevant])

# Combine English and Marathi data
df_combined = pd.concat([df_en_combined, df_mr_combined], ignore_index=True)

In [28]:
# Shuffle the combined DataFrame to ensure a random distribution of data
df_combined = df_combined.sample(frac=1).reset_index(drop=True)

In [29]:
len(df_combined)

12508

In [30]:
print(df_combined.isna().sum())

source_url    0
target_url    0
text          0
summary       0
relevance     0
dtype: int64


In [34]:
# Sampling 10,000 rows from each DataFrame
df_en_sample = df_en_test.sample(n=1000, random_state=42)  # English DataFrame
df_mr_sample = df_val_cs_mar.sample(n=1000, random_state=42)  # Marathi DataFrame

In [35]:
# Reset index to align the samples
df_en_sample.reset_index(drop=True, inplace=True)
df_mr_sample.reset_index(drop=True, inplace=True)

# Create cross-lingual irrelevant pairs
# English text with Marathi summary
df_en_mr_irrelevant = pd.DataFrame({
    'text': df_en_sample['text'],
    'summary': df_mr_sample['summary'],
    'relevance': 0  # Mark as irrelevant
})

# Marathi text with English summary
df_mr_en_irrelevant = pd.DataFrame({
    'text': df_mr_sample['text'],
    'summary': df_en_sample['summary'],
    'relevance': 0  # Mark as irrelevant
})

In [36]:
df_final_validation = pd.concat([relevance_dataset_2, df_en_mr_irrelevant, df_mr_en_irrelevant, df_combined])

In [37]:
len(df_final_validation)

19192

In [38]:
df_final_validation.columns

Index(['summary', 'text', 'relevance', 'source_url', 'target_url'], dtype='object')

In [18]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Ensure a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model_name = "october-sd/MuRIL_relevance"
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
model.eval()  # Put the model in evaluation mode

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [39]:
df_f_v = df_final_validation[['summary','text','relevance']]

In [40]:
print(df_f_v.isna().sum())

summary      0
text         0
relevance    0
dtype: int64


In [42]:
print(len(df_f_v))

19192


In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Prepare lists to store outputs
true_labels = []
predictions = []

# Inference
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for batch in val_loader:
        # Move batch to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass, calculate predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        pred_labels = torch.argmax(logits, dim=1)

        # Store predictions and true labels
        predictions.extend(pred_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9924
Precision: 0.9924
Recall: 0.9924
F1 Score: 0.9924


In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Prepare lists to store outputs
true_labels = []
predictions = []

# Inference
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for batch in val_loader:
        # Move batch to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass, calculate predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        pred_labels = torch.softmax(logits, dim=1)

        # Store predictions and true labels
        predictions.extend(pred_labels.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [ ]:
print(predictions)

In [50]:
print(logits)

tensor([[ 2.3244, -2.3004],
        [ 2.3241, -2.3000],
        [-2.0941,  2.0769],
        [ 2.3247, -2.3010],
        [ 2.3235, -2.2999],
        [-2.0944,  2.0775],
        [ 2.3231, -2.2985],
        [ 2.3248, -2.3011],
        [ 2.3228, -2.2984],
        [-2.0939,  2.0773],
        [-2.0938,  2.0771],
        [ 2.3238, -2.2999],
        [ 2.3246, -2.3002],
        [ 2.2837, -2.2590],
        [-2.0939,  2.0773],
        [-2.0940,  2.0771],
        [-2.0949,  2.0780],
        [-2.0941,  2.0771],
        [ 2.3224, -2.2975],
        [-2.0946,  2.0778],
        [ 2.3247, -2.3010],
        [ 2.1408, -2.1177],
        [ 2.3233, -2.2993],
        [-2.0949,  2.0779],
        [-2.0942,  2.0774],
        [ 2.3219, -2.2976],
        [ 2.3240, -2.3001],
        [-2.0948,  2.0778],
        [-2.0886,  2.0713],
        [-2.0946,  2.0778],
        [-2.0882,  2.0710],
        [ 2.3252, -2.3013],
        [ 2.3235, -2.2995],
        [-2.0941,  2.0776],
        [-2.0944,  2.0775],
        [-2.0946,  2